In [9]:
import os
import random

import numpy as np
from tqdm import tqdm
from PIL import Image

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt

import ssl
ssl._create_default_https_context = ssl._create_unverified_context
dinov2_vitl14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')

class GraspMask:
    def __init__(self):
        self.dino_v2 = torch.hub.load("facebookresearch/dinov2", "dinov2_vits14")
        self.transform =  transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Resize(256),
                            transforms.CenterCrop(224),
                            transforms.Normalize([0.485, 0.456, 0.406],
                                                [0.229, 0.224, 0.225])
                        ])
        
    def get_features(self, image_path) -> torch.Tensor:
        pass
        
        
    def get_mask(self, image_path):
        image = Image.open(image_path).convert("RGB")
        image = self.transform(image)
        image_features = self.dino_v2(image)
        
        
        return image_features

Using cache found in /Users/jacintosunersoler/.cache/torch/hub/facebookresearch_dinov2_main


In [10]:
grasp_mask = GraspMask()
mask = grasp_mask.get_mask("/Users/jacintosunersoler/Desktop/TFG/robot-grasping/images/IMG_1375.jpeg")

Using cache found in /Users/jacintosunersoler/.cache/torch/hub/facebookresearch_dinov2_main


<class 'PIL.Image.Image'>


RuntimeError: Numpy is not available

In [8]:
import torch
from PIL import Image
import torchvision.transforms as T

dinov2_vits14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')

img = Image.open('images/image_1.jpeg')

transform = T.Compose([
T.Resize(224),
T.CenterCrop(224),
T.ToTensor(),
T.Normalize(mean=[0.5], std=[0.5]),
])

img = transform(img)[:3].unsqueeze(0)

with torch.no_grad():
    features = dinov2_vits14.forward_features(img)["x_norm_patchtokens"]

print(features.shape)
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
pca.fit(features)

pca_features = pca.transform(features)
pca_features = (pca_features - pca_features.min()) / (pca_features.max() - pca_features.min())
pca_features = pca_features * 255

plt.imshow(pca_features.reshape(16, 16, 3).astype(np.uint8))
plt.savefig('meta_dog_features.png')

Using cache found in /Users/jacintosunersoler/.cache/torch/hub/facebookresearch_dinov2_main


torch.Size([1, 256, 384])


ModuleNotFoundError: No module named 'sklearn'

In [3]:
import numpy as np
camera_matrix = np.array([[613.32427146,  0.,        633.94909346],
       [ 0.,        614.36077155, 363.33858573],
       [ 0.,          0.,          1.       ]])

def process_rgbd_data(input_npy_path, output_npy_path, camera_matrix):
    # Load the 4D array from the .npy file
    data = np.load(input_npy_path, allow_pickle=True)

    # Assuming the input data shape is (H, W, C, D)
    # where C is the channel (BGRD)
    bgrd_data = data  # The loaded 4D array

    # Extract BGR and Depth data
    bgr_image = bgrd_data[..., :3]  # Extract the BGR channels
    depth_image = bgrd_data[..., 3]  # Extract the depth channel

    # Prepare the new data dictionary
    new_data_dict = {
        "bgr": bgr_image,
        "depth": depth_image,
        "K": camera_matrix  # Assume camera_matrix is passed as an argument
    }

    # Save the new dictionary as a .npy file
    np.save(output_npy_path, new_data_dict)

    print(f"Saved processed RGBD data to {output_npy_path}")


process_rgbd_data("/home/jacinto/robot-grasp/data/rgbds/cup_and_holder.npy", "/home/jacinto/robot-grasp/data/rgbds/processed_data.npy", camera_matrix)

Saved processed RGBD data to /home/jacinto/robot-grasp/data/rgbds/processed_data.npy
